# Convolution Network -----> food classifier

In [1]:
# Adapted notebook from Author: Aymeric Damien
# Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [1]:
#util and imports
%matplotlib inline

import os
import fnmatch
from cStringIO import StringIO
import numpy as np
import PIL.Image
import IPython.display
import shutil
import matplotlib.pyplot as plt


def list_all_files(directory, extensions=None):
    for root, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            base, ext = os.path.splitext(filename)
            joined = os.path.join(root, filename)
            if extensions is None or ext.lower() in extensions:
                yield joined
                
def show_array(a, fmt='png', filename=None):
    a = np.uint8(np.clip(a, 0, 255))
    image_data = StringIO()
    PIL.Image.fromarray(a).save(image_data, fmt)
    if filename is None:
        IPython.display.display(IPython.display.Image(data=image_data.getvalue()))
    else:
        with open(filename, 'w') as f:
            image_data.seek(0)
            shutil.copyfileobj(image_data, f)

def find_rectangle(n, max_ratio=2):
    sides = []
    square = int(math.sqrt(n))
    for w in range(square, max_ratio * square):
        h = n / w
        used = w * h
        leftover = n - used
        sides.append((leftover, (w, h)))
    return sorted(sides)[0][1]

# should work for 1d and 2d images, assumes images are square but can be overriden
def make_mosaic(images, n=None, nx=None, ny=None, w=None, h=None):
    if n is None and nx is None and ny is None:
        nx, ny = find_rectangle(len(images))
    else:
        nx = n if nx is None else nx
        ny = n if ny is None else ny
    images = np.array(images)
    if images.ndim == 2:
        side = int(np.sqrt(len(images[0])))
        h = side if h is None else h
        w = side if w is None else w
        images = images.reshape(-1, h, w)
    else:
        h = images.shape[1]
        w = images.shape[2]
    image_gen = iter(images)
    mosaic = np.empty((h*ny, w*nx))
    for i in range(ny):
        ia = (i)*h
        ib = (i+1)*h
        for j in range(nx):
            ja = j*w
            jb = (j+1)*w
            mosaic[ia:ib, ja:jb] = next(image_gen)
    return mosaic

In [2]:
arros = list(list_all_files('../img_classifier/arros/', ['.jpg']))
print 'loaded', len(arros), "exemples d'arròs"
carn = list(list_all_files('../img_classifier/carn_2/', ['.jpg']))
print 'loaded', len(carn), "exemples de carn"
entrepans = list(list_all_files('../img_classifier/entrepans/', ['.jpg']))
print 'loaded', len(entrepans), "exemples d'entrepans"
pasta = list(list_all_files('../img_classifier/pasta/', ['.jpg']))
print 'loaded', len(pasta), "exemples de pasta"
pizza = list(list_all_files('../img_classifier/pizza/', ['.jpg']))
print 'loaded', len(pizza), "exemples de pizza"
sopa = list(list_all_files('../img_classifier/sopa/', ['.jpg']))
print 'loaded', len(sopa), "exemples de sopa"
sushi = list(list_all_files('../img_classifier/sushi_2/', ['.jpg']))
print 'loaded', len(sushi), "exemples de sushi"
'''carne = list(list_all_files('../japanese_food/carn/', ['.jpg']))
print 'loaded', len(carne), "exemples de carne"'''
'''tempura = list(list_all_files('../../notebooks/projecte/UBdatascience/images_classifier/Tempura/', ['.jpg']))
print 'loaded', len(tempura), "exemples de tempura"'''
verd = list(list_all_files('../img_classifier/verd/', ['.jpg']))
print 'loaded', len(verd), "exemples de verd"
examples = [(path, 0) for path in arros]+[(path, 1) for path in carn]+[(path, 2) for path in entrepans] \
+[(path, 3) for path in pasta]+[(path, 4) for path in pizza]+[(path, 5) for path in sopa]\
+[(path, 6) for path in sushi]+[(path, 7) for path in verd]

loaded 6416 exemples d'arròs
loaded 7078 exemples de carn
loaded 5495 exemples d'entrepans
loaded 6474 exemples de pasta
loaded 3165 exemples de pizza
loaded 9171 exemples de sopa
loaded 6199 exemples de sushi
loaded 9290 exemples de verd


In [3]:
import numpy as np
from skimage.measure import block_reduce
from skimage.io import imread
from skimage import transform

def examples_to_dataset(examples, block_size=2):
    X = []
    y = []
    number = 0
    sum_img_mean = 0
    size = 112
    for path, label in examples:        
        try:
            img = imread(path) #, as_grey=True
            rows = img.shape[0]
            cols = img.shape[1]
            center = np.array([rows/2, cols/2])     

            if (rows>cols):
                crop_size = cols/2
                img = img[center[0]-crop_size:center[0]+crop_size,:,:]
            else : 
                crop_size = rows/2 
                img = img[:,center[1]-crop_size:center[1]+crop_size,:]

            img = transform.resize(img, (size,size))      
            img = block_reduce(img, block_size=(2, 2, 1), func=np.mean)
            img = img.reshape(56*56*3)
            X.append(img)

            if(label==0):
                y.append((1,0,0,0,0,0,0,0))
            elif(label==1):
                y.append((0,1,0,0,0,0,0,0))
            elif(label==2):
                y.append((0,0,1,0,0,0,0,0))
            elif(label==3):
                y.append((0,0,0,1,0,0,0,0))
            elif(label==4):
                y.append((0,0,0,0,1,0,0,0))
            elif(label==5):
                y.append((0,0,0,0,0,1,0,0))
            elif(label==6):
                y.append((0,0,0,0,0,0,1,0))
            else:
                y.append((0,0,0,0,0,0,0,1))
        except:
            pass
        
    return np.asarray(X), np.asarray(y)

%time X, Y = examples_to_dataset(examples)
X = np.asarray(X,dtype=np.float32)
Y = np.asarray(Y,dtype=np.int32)

CPU times: user 4min 3s, sys: 8.28 s, total: 4min 11s
Wall time: 11min 44s


In [4]:
X.shape

(53288, 9408)

In [5]:
# Split data into train and test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.05, random_state=2)

In [6]:
mean = np.mean(X_train, axis=1).mean()
X_train = X_train - mean
X_test = X_test - mean

In [7]:
import pandas as pd

In [8]:
data = pd.DataFrame(X_test)

In [9]:
labels = []
for i in range(y_test.shape[0]):
    if np.array_equal(y_test[i],(1,0,0,0,0,0,0,0)):
        labels.append(0)
    if np.array_equal(y_test[i],(0,1,0,0,0,0,0,0)):
        labels.append(1)
    if np.array_equal(y_test[i],(0,0,1,0,0,0,0,0)):
        labels.append(2)
    if np.array_equal(y_test[i],(0,0,0,1,0,0,0,0)):
        labels.append(3)
    if np.array_equal(y_test[i],(0,0,0,0,1,0,0,0)):
        labels.append(4)
    if np.array_equal(y_test[i],(0,0,0,0,0,1,0,0)):
        labels.append(5)
    if np.array_equal(y_test[i],(0,0,0,0,0,0,1,0)):
        labels.append(6)
    if np.array_equal(y_test[i],(0,0,0,0,0,0,0,1)):
        labels.append(7)

In [10]:
data['labels']=labels

In [11]:
test_arros = np.array(data[data['labels']==0].drop('labels',1))
test_carn = np.array(data[data['labels']==1].drop('labels',1))
test_entrepans = np.array(data[data['labels']==2].drop('labels',1))
test_pasta = np.array(data[data['labels']==3].drop('labels',1))
test_pizza = np.array(data[data['labels']==4].drop('labels',1))
test_sopa = np.array(data[data['labels']==5].drop('labels',1))
test_sushi = np.array(data[data['labels']==6].drop('labels',1))
test_verd = np.array(data[data['labels']==7].drop('labels',1))

In [12]:
test_arros_y = np.array([(1,0,0,0,0,0,0,0) for i in range(test_arros.shape[0])])
test_carn_y = np.array([(0,1,0,0,0,0,0,0) for i in range(test_carn.shape[0])])
test_entrepans_y = np.array([(0,0,1,0,0,0,0,0) for i in range(test_entrepans.shape[0])])
test_pasta_y = np.array([(0,0,0,1,0,0,0,0) for i in range(test_pasta.shape[0])])
test_pizza_y = np.array([(0,0,0,0,1,0,0,0) for i in range(test_pizza.shape[0])])
test_sopa_y = np.array([(0,0,0,0,0,1,0,0) for i in range(test_sopa.shape[0])])
test_sushi_y = np.array([(0,0,0,0,0,0,1,0) for i in range(test_sushi.shape[0])])
test_verd_y = np.array([(0,0,0,0,0,0,0,1) for i in range(test_verd.shape[0])])

In [13]:
X_train.shape

(50623, 9408)

In [14]:
X_test.shape

(2665, 9408)

In [15]:
#show some pics if in gray scale
#plt.imshow(255*make_mosaic(X[:len(pasta)], 8),cmap='gray') 

In [16]:
import tensorflow as tf

In [17]:
# Parameters
learning_rate = 0.0001
training_iters = 52000
batch_size = 10
display_step = 20

In [18]:
# Network Parameters
n_input = 9408 # MNIST data input (img shape: 28*28)
n_classes = 8 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

In [19]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [20]:
# Create AlexNet model
def conv2d(name, l_input, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(l_input, w, strides=[1, 1, 1, 1], 
                                                  padding='SAME'),b), name=name)

def max_pool(name, l_input, k):
    return tf.nn.max_pool(l_input, ksize=[1, k, k, 1], strides=[1, k, k, 1], 
                          padding='SAME', name=name)

def norm(name, l_input, lsize=4):
    return tf.nn.lrn(l_input, lsize, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name=name)

def alex_net(_X, _weights, _biases, _dropout):
    # Reshape input picture
    _X = tf.reshape(_X, shape=[-1, 56, 56, 3])

    # Convolution Layer
    conv1 = conv2d('conv1', _X, _weights['wc1'], _biases['bc1'])
    # Max Pooling (down-sampling)
    pool1 = max_pool('pool1', conv1, k=2)
    # Apply Normalization
    norm1 = norm('norm1', pool1, lsize=4)
    # Apply Dropout
    norm1 = tf.nn.dropout(norm1, _dropout)

    # Convolution Layer
    conv2 = conv2d('conv2', norm1, _weights['wc2'], _biases['bc2'])
    # Max Pooling (down-sampling)
    pool2 = max_pool('pool2', conv2, k=2)
    # Apply Normalization
    norm2 = norm('norm2', pool2, lsize=4)
    # Apply Dropout
    norm2 = tf.nn.dropout(norm2, _dropout)

    # Convolution Layer
    conv3 = conv2d('conv3', norm2, _weights['wc3'], _biases['bc3'])
    # Max Pooling (down-sampling)
    pool3 = max_pool('pool3', conv3, k=2)
    # Apply Normalization
    norm3 = norm('norm3', pool3, lsize=4)
    # Apply Dropout
    norm3 = tf.nn.dropout(norm3, _dropout)

    # Fully connected layer
    # Reshape conv3 output to fit dense layer input
    dense1 = tf.reshape(norm3, [-1, _weights['wd1'].get_shape().as_list()[0]]) 
    # Relu activation
    dense1 = tf.nn.relu(tf.matmul(dense1, _weights['wd1']) + _biases['bd1'], name='fc1')
    
    # Relu activation
    dense2 = tf.nn.relu(tf.matmul(dense1, _weights['wd2']) + _biases['bd2'], name='fc2') 

    # Output, class prediction
    out = tf.matmul(dense2, _weights['out']) + _biases['out']
    return out

In [21]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 3, 96],stddev=0.01)),
    'wc2': tf.Variable(tf.random_normal([5, 5, 96, 192],stddev=0.01)),
    'wc3': tf.Variable(tf.random_normal([5, 5, 192, 192],stddev=0.01)),
    'wd1': tf.Variable(tf.random_normal([7*7*192, 1024],stddev=0.01)),
    'wd2': tf.Variable(tf.random_normal([1024, 1024],stddev=0.01)),
    'out': tf.Variable(tf.random_normal([1024, 8],stddev=0.01))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([96],stddev=0.01)),
    'bc2': tf.Variable(tf.random_normal([192],stddev=0.01)),
    'bc3': tf.Variable(tf.random_normal([192],stddev=0.01)),
    'bd1': tf.Variable(tf.random_normal([1024],stddev=0.01)),
    'bd2': tf.Variable(tf.random_normal([1024],stddev=0.01)),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [22]:
# Construct model
pred = alex_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

In [23]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_xs, batch_ys = X_train[(batch_size*(step-1)):batch_size*(step)],y_train[(batch_size*(step-1)):batch_size*(step)]
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_xs,
                                                              y: batch_ys,
                                                              keep_prob: 1.})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"

    # Calculate accuracy for test images
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: X_test,
                                      y: y_test,
                                      keep_prob: 1.})
    print "Testing Accuracy for arros:", \
        sess.run(accuracy, feed_dict={x: test_arros,
                                      y: test_arros_y,
                                      keep_prob: 1.})
    print "Testing Accuracy for carn:", \
        sess.run(accuracy, feed_dict={x: test_carn,
                                      y: test_carn_y,
                                      keep_prob: 1.})
    print "Testing Accuracy for entrepans:", \
        sess.run(accuracy, feed_dict={x: test_entrepans,
                                      y: test_entrepans_y,
                                      keep_prob: 1.})
    print "Testing Accuracy for pasta:", \
        sess.run(accuracy, feed_dict={x: test_pasta,
                                      y: test_pasta_y,
                                      keep_prob: 1.})
    print "Testing Accuracy for pizza:", \
        sess.run(accuracy, feed_dict={x: test_pizza,
                                      y: test_pizza_y,
                                      keep_prob: 1.})
    print "Testing Accuracy for sopa:", \
        sess.run(accuracy, feed_dict={x: test_sopa,
                                      y: test_sopa_y,
                                      keep_prob: 1.})
    print "Testing Accuracy for sushi:", \
        sess.run(accuracy, feed_dict={x: test_sushi,
                                      y: test_sushi_y,
                                      keep_prob: 1.})
    print "Testing Accuracy for verd:", \
        sess.run(accuracy, feed_dict={x: test_verd,
                                      y: test_verd_y,
                                      keep_prob: 1.})

Iter 200, Minibatch Loss= 2.053283, Training Accuracy= 0.10000
Iter 400, Minibatch Loss= 2.028992, Training Accuracy= 0.20000
Iter 600, Minibatch Loss= 1.894882, Training Accuracy= 0.20000
Iter 800, Minibatch Loss= 2.106529, Training Accuracy= 0.10000
Iter 1000, Minibatch Loss= 1.755827, Training Accuracy= 0.40000
Iter 1200, Minibatch Loss= 2.057027, Training Accuracy= 0.40000
Iter 1400, Minibatch Loss= 1.944765, Training Accuracy= 0.30000
Iter 1600, Minibatch Loss= 1.991502, Training Accuracy= 0.10000
Iter 1800, Minibatch Loss= 1.818081, Training Accuracy= 0.30000
Iter 2000, Minibatch Loss= 2.087934, Training Accuracy= 0.20000
Iter 2200, Minibatch Loss= 1.716215, Training Accuracy= 0.30000
Iter 2400, Minibatch Loss= 2.221374, Training Accuracy= 0.10000
Iter 2600, Minibatch Loss= 1.671345, Training Accuracy= 0.60000
Iter 2800, Minibatch Loss= 1.882456, Training Accuracy= 0.40000
Iter 3000, Minibatch Loss= 1.857820, Training Accuracy= 0.20000
Iter 3200, Minibatch Loss= 1.811737, Trainin